In [ ]:
import numpy as np
import tensorflow as tf
import os
from datetime import datetime

In [ ]:
# params={}
num_inputs=28*28
num_hidden_layers=2
num_neurons=[600,500,400,200]
n_outputs=10
learning_rate=0.001
dropout_rate=0.5
activation_func=tf.nn.elu
initializer=tf.contrib.layers.xavier_initializer
batch_mormalization_momentum=0.9
optimizer_fn=tf.train.AdamOptimizer

n_epochs=100
batch_size=150


now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

savedir='/tmp/my_model_final_new.ckpt'


In [ ]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
X_test=mnist.test.images.reshape((-1,28*28))
y_test=mnist.test.labels

In [ ]:
def build_DNN_graph():
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,num_inputs])
    Y=tf.placeholder(tf.int32,[None])
    training=tf.placeholder_with_default(False,shape=(),name='training')
    inputs=X
    global_step=tf.Variable(0,trainable=False,name='global_step')
    
    for i in range(num_hidden_layers):
        inputs_before_act=tf.layers.dense(inputs,num_neurons[i],name='hidden_'+str(i))
        batch_normalized_inputs=tf.layers.batch_normalization(inputs_before_act,training=training,momentum=batch_mormalization_momentum)
        inputs=activation_func(batch_normalized_inputs,name='activation_function_'+str(i))
        inputs=tf.layers.dropout(inputs,dropout_rate,training=training)
        
    logits=tf.layers.dense(inputs,n_outputs,name='logits')
    entropies=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='entropies')
    loss=tf.reduce_mean(entropies,name='loss')
    optimizer=optimizer_fn(learning_rate)
    training_op=optimizer.minimize(loss,global_step=global_step)
    
    correct=tf.nn.in_top_k(logits,Y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

    extra_update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    init=tf.global_variables_initializer()
    
    loss_summary=tf.summary.scalar('loss',loss)
    acc_summary=tf.summary.scalar('accuracy',accuracy)
    summaries=tf.summary.merge_all()
    file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
    saver=tf.train.Saver()
    return X,Y,training,init,training_op,extra_update_ops,global_step,summaries,file_writer,saver
    

In [ ]:
X,Y,training,init,training_op,extra_update_ops,global_step,summaries,file_writer,saver=build_DNN_graph()
with tf.Session()as sess: 
#     if os.path.isfile(savedir):
#         saver.restore(sess,savedir)
#     else:
#         init.run()
    init.run()
    for epoch in range(n_epochs):
        print ("epoch"+str(epoch))
        for iteration in range(mnist.train.num_examples//batch_size):
            step=global_step.eval()
#             print ("Iteration "+str(iteration))
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            X_batch=X_batch.reshape(-1,num_inputs)
            
#             step=epoch*(mnist.train.num_examples//batch_size)+iteration
            if iteration%10==0:
                summary_str=summaries.eval(feed_dict={X:X_test,Y:y_test,training:False})
                file_writer.add_summary(summary_str,step)

            sess.run([training_op],feed_dict={X:X_batch,Y:y_batch,training:True})
#             print ('iter_loss',iloss)
#         acc_train=accuracy.eval(feed_dict={X:X_batch,Y:y_batch,training:False})
#         acc_test=accuracy.eval(feed_dict={X:X_test,Y:y_test,training:False})
#         print(epoch,"Train accuracy:",acc_train,"Test accuracy:",acc_test)
            
        if (epoch%5)==0:# checkpoint every 5 epochs
            save_path=saver.save(sess,savedir)
    file_writer.close()

    

In [ ]:
# print ('iterloss',3)